In [ ]:
import openpyxl
import pandas as pd
import glob
import os
import re
from openpyxl import load_workbook
from openpyxl.styles import Alignment
from openpyxl.styles import Alignment, Border, Side, Font, PatternFill


summary_dir=r'C:\Users\fbpza\Desktop\My_utils\Summary files'
BOQ_file_path=r'C:\Users\fbpza\Desktop\My_utils\Peshawar Highway 02 - BoQ.xlsx'
output_file_path = r'C:\Users\fbpza\Desktop\My_utils\Peshawar Highway 02 Master sheet BoQ_Updated.xlsx'

In [ ]:
def format_segment_name(segment_name):
    match = re.match(r'Segment (\d+)([A-Z]*)', segment_name)
    if match:
        number_part = match.group(1)
        alpha_part = match.group(2)
        formatted_number_part = str(int(number_part))
        return formatted_number_part + alpha_part
    return segment_name  # Return as is if no match

In [ ]:
summ_files = glob.glob(os.path.join(summary_dir, '*.csv'))
roads_dict = {}

for summ_file in summ_files:
    road_name = os.path.basename(summ_file).split('_Summary')[0]
    summ_df = pd.read_csv(summ_file)
    
    # Create a dictionary for segments with PCI and Width values
    segments_dict = summ_df.set_index('Segments')[['PCI', 'Width']].to_dict(orient='index')
    
    # Create a formatted dictionary with the required structure
    formatted_segments_dict = {format_segment_name(seg): {'PCI': data['PCI'], 'Width': data['Width']} for seg, data in segments_dict.items()}
    
    roads_dict[road_name] = formatted_segments_dict

boq_workbook = openpyxl.load_workbook(BOQ_file_path)
BOQ_DF = pd.ExcelFile(BOQ_file_path)
boq_sheets = BOQ_DF.sheet_names
# print(len(boq_sheets))
for sheet in boq_sheets:
    cost_df = pd.read_excel(BOQ_file_path, sheet_name=sheet, header=2)
    cost_df_header0 = pd.read_excel(BOQ_file_path, sheet_name=sheet, header=0)   

    rd_name_inBOQ = cost_df_header0.columns[0].strip()
    # print(f"Processing sheet: {rd_name_inBOQ}")

   
    S_no = cost_df['S/No'].tolist()
    PCI_column = cost_df['PCI Value / Road Classification']
    length_in_meters = cost_df['Length'].tolist()

    # Replace "Asphalt" with the PCI value
    updated_pci_column = []
    updated_width_column = []
    
    
    for each_segNo in S_no:
        
        segindex  = S_no.index(each_segNo)
        pci_value = PCI_column[segindex]
        each_segNo= str(each_segNo)
        
        width_value='- -'
        
        if pci_value == 'Asphalt':
            seg_number=each_segNo.lstrip()
            seg_number=re.sub(r'^0+', '', seg_number)
            seg_full_name = seg_number
            if rd_name_inBOQ in roads_dict and seg_full_name in roads_dict[rd_name_inBOQ]:
                pci_value = roads_dict[rd_name_inBOQ][seg_full_name]['PCI']
                width_value = roads_dict[rd_name_inBOQ][seg_full_name]['Width']
            else:
                print(rd_name_inBOQ)
                # print(f"Segment {seg_full_name} not found in roads_dict or has no PCI value.")
        
        updated_pci_column.append(pci_value)
        updated_width_column.append(width_value)
    
    # Update the DataFrame with the new PCI values
    cost_df['PCI Value / Road Classification'] = updated_pci_column
    cost_df['Width'] = updated_width_column
    new_row = [rd_name_inBOQ,'','','','','','','','','']
    newdf=pd.DataFrame([new_row], columns=cost_df.columns)
    # Convert the DataFrame to start with the new value
    df_with_new_value = pd.concat([newdf, cost_df], ignore_index=True)
    cost_df=df_with_new_value

    
    # Save the updated DataFrame back to the Excel sheet
    with pd.ExcelWriter(output_file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            cost_df.to_excel(writer, sheet_name=sheet, index=False)

excel_file = pd.ExcelFile(output_file_path)

sheets_dict = {}

for sheet_name in excel_file.sheet_names:
    df = pd.read_excel(excel_file, sheet_name=sheet_name,header=None)
    
    df.iloc[[0, 1]] = df.iloc[[1, 0]]
    
    sheets_dict[sheet_name] = df

with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
    for sheet_name, df in sheets_dict.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False, header=None)
    
        
    

print("Processing complete.")


In [ ]:
thin_border = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin'))

fill_color = PatternFill(start_color="CCFFCC", end_color="CCFFCC", fill_type="solid")
 
   
workbook = load_workbook(filename=output_file_path)
# Iterate through each sheet
for sheet_name in workbook.sheetnames:
    sheet = workbook[sheet_name]
    max_column = sheet.max_column
    max_row = sheet.max_row

    # Road Name cell setting
    sheet.merge_cells(start_row=1, start_column=1, end_row=1, end_column=max_column)
    rd_name_cells = sheet.cell(row=1, column=1)
    rd_name_cells.font = Font(bold=True, size=13)
    rd_name_cells.alignment = Alignment(horizontal='center', vertical='center')
    rd_name_cells.fill = fill_color
    rd_name_cells.border = thin_border

    # Header Row cells setting
    for row in sheet['A2:J2']:
        for cell in row:
            cell.border = thin_border
            cell.font = Font(bold=True, size=11)
            cell.alignment = Alignment(horizontal='center', vertical='center')
            cell.fill = fill_color
    
    # Data Rows cells setting
    for row in sheet[f'A3:J'+str(max_row)]:
        for cell in row:
            cell.border = thin_border
            cell.font = Font(bold=True, size=11)
            cell.alignment = Alignment(horizontal='center', vertical='center')
            cell.fill = PatternFill(start_color="FCFFCC", end_color="FFFFCC", fill_type="solid")    
        

workbook.save(output_file_path)
print("Alignment DONE")